# 보정 요약 결과 생성 (Correction Summary)

> **Purpose:** LLM-as-a-Judge 평가 결과를 기반으로 AI SOAP 노트의 보정 요약을 생성합니다.
> - Judge 점수 < 4.0인 SOAP 노트를 대상으로 보정 실행
> - 보정 결과를 `P2T2.ai_results.correction_summary` 테이블에 저장
>
> **카탈로그:** `P2T2`

## 0. 카탈로그 설정

In [ ]:
spark.sql("USE CATALOG P2T2")
print("✅ Catalog: P2T2")

## 1. 라이브러리 및 설정

In [ ]:

import openai
import json
from pyspark.sql import functions as F

# Azure OpenAI
CORRECTION_API_KEY = dbutils.secrets.get(scope="key-vault-scope", key="openai-api-key")
CORRECTION_ENDPOINT = dbutils.secrets.get(scope="key-vault-scope", key="openai-endpoint")

client = openai.AzureOpenAI(
    azure_endpoint=CORRECTION_ENDPOINT,
    api_key=CORRECTION_API_KEY,
    api_version="2025-01-01-preview"
)

CORRECTION_MODEL = "gpt-51-deploy"
print("✅ Correction 설정 완료")

## 2. 보정 프롬프트

In [ ]:

CORRECTION_PROMPT = """
Based on the following LLM-as-a-Judge evaluation, generate a corrected and improved 
clinical summary. Focus on fixing identified issues and enhancing clarity.

## Original SOAP Note:
{original_soap}

## Judge Evaluation:
{judge_evaluation}

## Corrections Needed:
{corrections_needed}

## Instructions:
1. Address all identified corrections
2. Maintain medical accuracy
3. Keep actionable recommendations
4. Write concise ER physician-ready format
5. Include confidence level

Output a corrected clinical summary (max 500 words).
"""

print("✅ 보정 프롬프트 정의 완료")

## 3. 보정 요약 생성

In [ ]:

def generate_correction(patient_id, original_soap, judge_evaluation):
    """
    Judge 평가를 반영한 보정 요약을 생성합니다.
    """
    corrections = judge_evaluation.get("corrections_needed", [])
    
    prompt = CORRECTION_PROMPT.format(
        original_soap=original_soap,
        judge_evaluation=json.dumps(judge_evaluation, ensure_ascii=False, default=str),
        corrections_needed="\n".join(f"- {c}" for c in corrections) if corrections else "No major corrections.",
    )
    
    try:
        response = client.chat.completions.create(
            model=CORRECTION_MODEL,
            messages=[
                {"role": "system", "content": "Medical document editor. Accuracy and clarity."},
                {"role": "user", "content": prompt},
            ],
            max_completion_tokens=800,
            temperature=0.2,
        )
        summary = response.choices[0].message.content or "[Content filtered]"
        overall_score = judge_evaluation.get("overall_score", 0.0)
        confidence = judge_evaluation.get("confidence", 0.0)
        adjusted = min(1.0, (overall_score / 5.0 + confidence) / 2)
        
        return {"patient_id": patient_id, "summary": summary, "confidence": str(round(adjusted, 3))}
    except Exception as e:
        return {"patient_id": patient_id, "summary": f"Error: {str(e)}", "confidence": "0.0"}

print("✅ 보정 요약 함수 준비 완료")

## 4. 배치 보정 및 저장

In [ ]:

def batch_correction_summary():
    """
    Judge 점수 < 4.0인 SOAP 노트를 보정하여 저장합니다.
    """
    df = spark.sql("""
        SELECT 
            j.patient_id,
            j.evaluation_json,
            j.overall_score,
            s.soap_note
        FROM P2T2.ai_results.judge_evaluation j
        JOIN P2T2.ai_results.openai_soap_notes s ON j.patient_id = s.patient_id
        WHERE CAST(j.overall_score AS DOUBLE) < 4.0
        ORDER BY CAST(j.overall_score AS DOUBLE) ASC
    """)
    
    rows = df.collect()
    results = []
    
    print(f"📝 보정 대상: {len(rows)}건 (Judge < 4.0)")
    
    for row in rows:
        evaluation = json.loads(row["evaluation_json"])
        result = generate_correction(
            patient_id=row["patient_id"],
            original_soap=row["soap_note"],
            judge_evaluation=evaluation,
        )
        results.append(result)
    
    if results:
        df_results = spark.createDataFrame(results)
        df_results = df_results.withColumn("processed_at", F.current_timestamp())
        
        df_results.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable("P2T2.ai_results.correction_summary")
        
        print(f"✅ 보정 저장: {len(results)}건 → P2T2.ai_results.correction_summary")
    else:
        print("ℹ️ 보정 대상 없음 (모든 SOAP 점수 ≥ 4.0)")

batch_correction_summary()